## Set Index of `DataFrame`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
%run lib/ingest_data.py

### Dataframes


In [ ]:
breast_cancer_df.head()

In [ ]:
breast_cancer_df.shape

In [ ]:
type(breast_cancer_df)

### Set Patient ID as Index

In [ ]:
try:
    breast_cancer_df.set_index('Sample_code_number', verify_integrity=True)
except ValueError as e:
    print("ValueError: " + str(e))

#### For a Duplicate Index What are the Counts?

In [ ]:
group_by_sample_code = breast_cancer_df.groupby('Sample_code_number')
group_by_sample_code.aggregate('count').Diagnosis.unique()

#### `pd.DataFrame.drop_duplicates()`

This will drop exact duplicate rows.

In [ ]:
breast_cancer_df.drop_duplicates().shape

In [ ]:
breast_cancer_df.drop_duplicates(inplace=True)

But is insufficient to fix the problem

In [ ]:
try:
    breast_cancer_df.set_index('Sample_code_number', verify_integrity=True)
except ValueError as e:
    print("ValueError: " + str(e))

In [ ]:
duplicate_mask = breast_cancer_df.duplicated('Sample_code_number')
duplicate_mask.head(10)

In [ ]:
breast_cancer_df[duplicate_mask].shape

In [ ]:
duplicate_sample_ids = breast_cancer_df[duplicate_mask].Sample_code_number.unique()

In [ ]:
duplicate_sample_ids.shape

In [ ]:
duplicate_sample_ids

In [ ]:
duplicate_mask_2 = breast_cancer_df.Sample_code_number.isin(duplicate_sample_ids)

In [ ]:
breast_cancer_df[duplicate_mask_2].shape

In [ ]:
group_by_sample_code.aggregate('count').Diagnosis.head()

In [ ]:
group_by_sample_code = breast_cancer_df.groupby('Sample_code_number')
group_by_sample_code.aggregate('count').Diagnosis.unique()
sample_code_counts = group_by_sample_code.aggregate('count').Diagnosis

In [ ]:
(sample_code_counts > 1).head()

In [ ]:
sample_code_counts[sample_code_counts > 1].index

In [ ]:
sample_code_counts[sample_code_counts > 1].shape

In [ ]:
manual_duplicate_indices = sample_code_counts[sample_code_counts > 1].index

In [ ]:
manual_duplicate_mask = breast_cancer_df.Sample_code_number.isin(manual_duplicate_indices)

In [ ]:
breast_cancer_df[manual_duplicate_mask].shape

In [ ]:
repeated_samples = breast_cancer_df[manual_duplicate_mask].sort_values('Sample_code_number')

In [ ]:
repeated_samples.head(10)

In [ ]:
repeated_samples.groupby(['Sample_code_number', 'group'])[['Diagnosis']].count().head(10)

In [ ]:
group_sample_count = repeated_samples.groupby(['Sample_code_number', 'group']).count()

In [ ]:
group_sample_count[group_sample_count['Diagnosis'] > 1]

In [ ]:
breast_cancer_df[breast_cancer_df.Sample_code_number == 493452]

In [ ]:
breast_cancer_df['id'] = breast_cancer_df.Sample_code_number.astype(str) + '_' + breast_cancer_df.group

In [ ]:
breast_cancer_df.head()

In [ ]:
breast_cancer_df.groupby('id').agg('count')['Diagnosis'].unique()

In [ ]:
id_count = breast_cancer_df.groupby('id').agg('count')['Diagnosis']

In [ ]:
id_count[id_count > 1]

In [ ]:
repeated_ids = id_count[id_count > 1].index

In [ ]:
breast_cancer_df[breast_cancer_df.id.isin(repeated_ids)].sort_values('id').head(10)

In [ ]:
breast_cancer_df[breast_cancer_df.id.isin(repeated_ids)].sort_values('id').id

In [ ]:
test_ids = ['1','2','2','3','3','3','4','5']

In [ ]:
counts = {}
new_test_ids = []
for ID in test_ids:
    if ID not in counts.keys():
        counts[ID] = 1
        new_test_ids.append(ID)
    else:
        counts[ID] += 1
        new_test_ids.append(ID+'_'+str(counts[ID]))
new_test_ids

In [ ]:
counts = {}
new_ids = []
for ID in breast_cancer_df.id:
    if ID not in counts.keys():
        counts[ID] = 1
        new_ids.append(ID)
    else:
        counts[ID] += 1
        new_ids.append(ID+'_'+str(counts[ID]))

In [ ]:
breast_cancer_df.id = new_ids

In [ ]:
breast_cancer_df.head(20)

In [ ]:
breast_cancer_df.set_index('id', inplace=True)

In [ ]:
breast_cancer_df.head()

In [ ]:
breast_cancer_df.drop(['Sample_code_number', 'group'], axis=1, inplace=True)

### Export to CSV

Ultimately, we will export a CSV of the dataframe to disk. This will make it easy to access the same data from both Python and R.


In [ ]:
%ls

In [ ]:
%mkdir -p data

In [ ]:
%ls

In [ ]:
breast_cancer_df.to_csv('data/breast_cancer.csv', index=False)